In [12]:
import pandas as pd
import numpy as np

from bokeh.models import BoxAnnotation, FactorRange
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral4
from bokeh.io import output_notebook

In [8]:
# Define the data
# Define the data
data = {
    'Gene': np.repeat(['Gene1', 'Gene2', 'Gene3', 'Gene4', 'Gene5'], 4),
    'veh': np.random.randint(0, 11, size=20),
    'oAB': np.random.randint(0, 11, size=20),
    'gsk': np.random.randint(0, 11, size=20),
    'oAB-gsk': np.random.randint(0, 11, size=20)
}

# Create the DataFrame
df = pd.DataFrame(data)

# Create the DataFrame
df = pd.DataFrame(data)

In [10]:
# Reshape the DataFrame
df_melted = df.melt(id_vars='Gene', var_name='Series', value_name='Values')

In [35]:
from bokeh.models import Whisker, ColumnDataSource

# Create a list of series names (will be used for the legend)
series_names = df_melted['Series'].unique().tolist()

# Create a list of unique combinations of 'Gene' and 'Series'
factors = df_melted[['Gene', 'Series']].drop_duplicates().values.tolist()

# Convert each combination to a tuple
factors = [tuple(x) for x in factors]

# Create a new figure with FactorRange for x-axis to handle the multi-index
p = figure(x_range=FactorRange(*factors), height=350, title="Grouped Dot Plot",
           toolbar_location=None, tools="")

# Add a dot plot for each series
for i, series_name in enumerate(series_names):
    df_filtered = df_melted[df_melted['Series'] == series_name]
    p.circle(x=[(gene, series_name) for gene in df_filtered['Gene']], y=df_filtered['Values'], 
             fill_color=Spectral4[i % len(Spectral4)], size=10, legend_label=series_name)

    # Calculate mean and standard deviation for each gene in the series
    means = df_filtered.groupby('Gene')['Values'].mean()
    stds = df_filtered.groupby('Gene')['Values'].std()

    # Create a ColumnDataSource that contains the mean and standard deviation
    source = ColumnDataSource(data=dict(
        base=[(gene, series_name) for gene in means.index],
        lower=means - stds,
        upper=means + stds
    ))

    # Add error bars
    p.add_layout(
        Whisker(source=source, base="base", upper="upper", lower="lower")
    )

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = 1.2
p.legend.location = "top_left"

# Show the plot
show(p)